In [1]:
import pandas as pd
import numpy as np

In [2]:
output_names = ['beckerestimation', 'csopII', 'DAT', 'gurcay2015estimation', 'jury', 'jury_TINY', 'pgg']

In [9]:
directory="../output/"
for dataset in output_names:
    csv_path = directory + dataset + '_output_conversation_level.csv'
    df = pd.read_csv(csv_path)
    print(dataset + ": " + str(df['discursive_diversity'].mean()))
    # mean_df = pd.DataFrame({'dataset': dataset, 'discursive mean': df['discursive_diversity'].mean()})

    # mean_df.to_csv('step1_DD_means.csv')



beckerestimation: 0.5549970523316429
csopII: 0.643433754406302
DAT: 0.5958520491707279
gurcay2015estimation: 0.5945824035668794
jury: 0.4269552462902284
jury_TINY: 0.3846901364054913
pgg: 0.5845007521581473
